In [ ]:
from keras import backend as bknd
from keras.callbacks import *
from keras.layers import *
from keras.models import *
from keras.optimizers import SGD
from keras.utils import *

from keras.callbacks import ModelCheckpoint
from keras.callbacks import TensorBoard
#from stn import spatial_transformer_network as transformer

import numpy as np
import keras
import numpy as np
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam,SGD
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D , Flatten,concatenate 
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras import metrics

from sklearn.utils import class_weight
from collections import Counter

import matplotlib.pyplot as plt

from os import listdir
from os.path import isfile, join

import pandas as pd
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

from keras.callbacks import EarlyStopping
import keras,os
from keras.preprocessing.image import ImageDataGenerator
import numpy as np

In [ ]:
train_loc = 'CWT\\train_4000'
test_loc = 'CWT\\clean\\test'
inputShape = Input((224,224, 3))  # base on Tensorflow backend
conv_1 = Conv2D(64, (3, 3), activation='relu', padding='same')(inputShape)
x_inputi = BatchNormalization()(conv_1)
conv_2 = Conv2D(64, (3, 3), activation='relu', padding='same')(x_inputi)
conv_3 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv_2)
batchnorm_3 = BatchNormalization()(conv_3)
x_inputi= MaxPooling2D(pool_size=(2, 2))(batchnorm_3)
conv_4 = Conv2D(64, (3, 3), activation='relu', padding='same')(x_inputi)
conv_5 = Conv2D(96, (5, 5), activation='relu', padding='same')(conv_4)
batchnorm_5 = BatchNormalization()(conv_5)
x_inputi= MaxPooling2D(pool_size=(2, 2))(batchnorm_5)
x_in1i = Conv2D(filters = int(x_inputi.shape[-1]//4), kernel_size = (1,1), strides = (1,1), padding = 'valid', activation = 'relu')(x_inputi)
x_in1i=BatchNormalization()(x_in1i)
x_in2i = Conv2D(filters = int(x_inputi.shape[-1]//2), kernel_size = (1,1), strides = (1,1), padding = 'valid', activation = 'relu')(x_in1i)
x_in2i=BatchNormalization()(x_in2i)
x_in3i = Conv2D(filters = int(x_inputi.shape[-1]//2), kernel_size = (3,3), strides = (1,1), padding = 'same', activation = 'relu')(x_in1i)
x_in3i=BatchNormalization()(x_in3i)
out = Concatenate() ([x_in2i, x_in3i])
conv_6 = Conv2D(96, (5, 5), activation='relu', padding='same')(out)
conv_7 = Conv2D(96, (5, 5), activation='relu', padding='same')(conv_6)
batchnorm_7 = BatchNormalization()(conv_7)
#pool_5 = MaxPooling2D(pool_size=(2, 2))(batchnorm_7)
bn_shape = batchnorm_7.get_shape()
print(bn_shape)  # (?, 50, 7, 512)

# reshape to (batch_size, width, height*dim)
# x_reshape = Reshape(target_shape=(int(bn_shape[1]), int(bn_shape[2] * bn_shape[3])))(stn_7)
x_reshape = Reshape(target_shape=(int(bn_shape[1]), int(bn_shape[2] * bn_shape[3])))(batchnorm_7)

fc_1 = Dense(128, activation='relu')(x_reshape)  # (?, 50, 128)

print(x_reshape.get_shape())  # (?, 50, 3584)
print(fc_1.get_shape())  # (?, 50, 128)

rnn_1 = LSTM(32, kernel_initializer="he_normal", return_sequences=True)(fc_1)
rnn_1b = LSTM(32, kernel_initializer="he_normal", go_backwards=True, return_sequences=True)(fc_1)
rnn1_merged = add([rnn_1, rnn_1b])

rnn_2 = LSTM(32, kernel_initializer="he_normal", return_sequences=True)(rnn1_merged)
rnn_2b = LSTM(32, kernel_initializer="he_normal", go_backwards=True, return_sequences=True)(rnn1_merged)
rnn2_merged = concatenate([rnn_2, rnn_2b])

drop_1 = Dropout(0.25)(rnn2_merged)
classifier=Flatten()(drop_1)
#fc_2 = Dense(512, kernel_initializer='he_normal', activation='relu')(classifier)
fc_2 = Dense(256, kernel_initializer='he_normal', activation='relu')(classifier)
fc_2 = Dense(128, kernel_initializer='he_normal', activation='relu')(classifier)
fc_2 = Dense(64, kernel_initializer='he_normal', activation='relu')(fc_2)
fc_2 = Dense(32, kernel_initializer='he_normal', activation='relu')(fc_2)
fc_2 = Dense(16, kernel_initializer='he_normal', activation='relu')(fc_2)
fc_2 = Dense(5, kernel_initializer='he_normal', activation='softmax')(fc_2)

# model setting
base_model = Model(inputs=inputShape, outputs=fc_2) 
adam = keras.optimizers.Adam(lr = 0.0001)
base_model.compile(loss = 'categorical_crossentropy', optimizer = adam, metrics = ['accuracy'])
trdata = ImageDataGenerator(rescale=1./255)
traindata = trdata.flow_from_directory(directory=train_loc, target_size=(224,224),batch_size=16)
tsdata = ImageDataGenerator(rescale=1./255)
testdata = tsdata.flow_from_directory(directory=test_loc, target_size=(224,224),batch_size=16)


filepath="CWT//cwt_weights//model_0_cwt_v2_awgn_trial.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
es = EarlyStopping(monitor='val_accuracy', mode='max', verbose=1, patience=20)
callbacks_list = [checkpoint,es]
base_model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001),loss=keras.losses.categorical_crossentropy,metrics=['accuracy'])
hist = base_model.fit_generator(epochs=60,generator=traindata,validation_data=testdata,callbacks=callbacks_list)

In [ ]:
model = keras.models.load_model("CWT//cwt_weights//model_0_cwt_v2_awgn_trial.h5")

In [ ]:
hisDF=pd.DataFrame(hist.history)
hisDF.to_csv('CWT//history_model_0_cwt_v2_trial.csv')